# Operacja na tabelach za pomocą pakietu Pandas

Na bazie danych z tabeli FCP do tabeli TKT dołączam 3 nowe kolumny (klucz: TICKET_NUMBER). 
Poniżej opis kolumn.

•	AIRPORT_PATH - lista lotnisk podróży w kolejności odbywania podróży

•	NIGHT_DEPARTURE - zmienna binarna, przyjmująca wartość 1 dla pasażerów rozpoczynających swoją podróż pomiędzy godz. 19.00 a 5:00

•	PRICE_KM_PLN  – cena w złotówkach za kilometr podróży (FLIGHT_DISTANCE)

In [1]:
import pandas as pd
import numpy as np

Powyższa instrukcja służy do wczytania bibliotek używanych w programie

Poniżej natomiast wczytuję bazę danych do zmiennej.

In [2]:
dataframe_for_work = pd.read_csv(r"FCP.csv", sep=';')

Sortuję dataframe, na której będę pracował. Następnie resetuje indexy i sprawdzam czy wszystko wykonało się poprawnie.

In [3]:
dataframe_for_work.sort_values(by=["TICKET_NUMBER","COUPON_NUMBER"],inplace=True)
dataframe_for_work.reset_index(drop=True,inplace=True)
dataframe_for_work.head()

,TICKET_NUMBER,COUPON_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,FLIGHT_DATE_LOCAL,TIME_DEPARTURE_LOCAL_TIME,FLIGHT_DISTANCE,FLIGHT_RANGE,MARKETING_CARRIER,OPERATIONAL_CARRIER,BOOKED_CLASS,BOOKED_CABIN,AIRCRAFT_TYPE,FARE_BASIS,VAB
0,125352212788,1,KLI,OUO,2007-06-08,10:35:00,918,SHORT-HAUL,2434615205489683,2434615205489683,O,Economy,735,OPRO18,OTHER
1,125352212788,2,OUO,TIL,2007-06-08,15:10:00,8598,LONG-HAUL,2434615205489683,2434615205489683,O,Economy,788,OPRO18,BASIC
2,125352212788,3,TIL,OUO,2007-07-03,10:15:00,8598,LONG-HAUL,2434615205489683,2434615205489683,U,Economy,788,ULTANEU,FLEX
3,125352212788,4,OUO,KLI,2007-07-03,17:15:00,918,SHORT-HAUL,2434615205489683,2434615205489683,U,Economy,735,ULTANEU,OTHER
4,176790448325,1,LPF,OUO,2007-04-06,18:25:00,523,SHORT-HAUL,2434615205489683,2434615205489683,U,Economy,E75,USAVI7,SAVER


In [4]:
dataframe_basic = pd.read_csv(r"TKT.csv", sep=';')
dataframe_basic.head()

,BOOKING_ID,TICKET_NUMBER,ORIGINAL_TICKET_NUMBER,FORM_OF_PAYMENT,CURRENCY,TOTAL_PRICE,TOTAL_PRICE_PLN,PAX_GENDER,PAX_TYPE,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,LOYAL_CUSTOMER_ID,LOYAL_CUSTOMER_DATE_OF_BIRTH,LOYAL_CUSTOMER_REGISTERED_DATE
0,1064066564877934,4488275897136592,9379265972706784,CA,EUR,135.87,565.86,M,ADULT,N,N,NaN,NaN,NaN
1,3802019702506290,8736272083689777,9379265972706784,CA,PLN,674.99,674.99,M,ADULT,N,N,NaN,NaN,NaN
2,8101801230833636,3984954741380444,9379265972706784,CA,PLN,986.26,986.26,M,ADULT,N,N,NaN,NaN,NaN
3,5917447822742897,8027575854357722,9379265972706784,CCVI4,PLN,469.73,469.73,M,ADULT,N,N,NaN,NaN,NaN
4,3802019702506290,6709320598486400,9379265972706784,CA,PLN,674.99,674.99,M,ADULT,N,N,NaN,NaN,NaN


Przypisuje kolumny, których będę potem używał do zmiennych.

In [5]:
tickets = dataframe_for_work.TICKET_NUMBER
flight_start =  dataframe_for_work.ORIGIN_AIRPORT_CODE
flight_end = dataframe_for_work.DESTINATION_AIRPORT_CODE

cost = dataframe_basic.TOTAL_PRICE_PLN
flight_distance = dataframe_for_work.FLIGHT_DISTANCE

Poniżej algorytm który sprawdza godzine i zwraca odpowiednią wartość (0, lub 1) w zależności o której godzinie rozpoczął się lot.

In [6]:
x = 0
tickets_list = []
night_departure = []

for i in tickets:
    number = i
    if number not in tickets_list:
        tickets_list.append(number)
        time = dataframe_for_work.TIME_DEPARTURE_LOCAL_TIME[x]
        if time < "19:00:00" and time > "05:00:00":
            dictionary = {0:number}
            night_departure.append(dictionary)
        if  time > "19:00:00" or time < "05.00:00":
            dictionary = {1:number}
            night_departure.append(dictionary)
    x += 1

Poniższy kod zamienia słownik na dataframe

In [7]:
output = pd.DataFrame()
df_dictionary = pd.DataFrame([night_departure])
output = pd.concat([output, df_dictionary], ignore_index=True)
frame = pd.DataFrame.from_dict(night_departure)

Sortuje i usuwam duplikaty biletów, zostawiając tylko ten bilet dla którego dany lot był pierwszym lotem. Usuwam zbędną kolumnę zero, następnie dodaje nową kolumnę "NIGHT_DEPARTURE" i przypisuje jej wartość do każdego wiersza równą 1. Bilety, gdzie NIGHT_DEPARTURE wynosi 0, zostaną z tej dataframe całkowicie usunięte. Później wartość 1 dopasuję do odpowiednich biletów.

In [8]:
frame['NIGHT_DEPARTURE '] = '1'
frame.rename(columns = {1 : 'TICKET_NUMBER'}, inplace = True)
frame = frame.drop(columns=[0])
frame = frame.sort_values(by = "TICKET_NUMBER", na_position="last")
frame = frame.drop_duplicates(subset="TICKET_NUMBER", keep='first', inplace=False)
frame.reset_index(drop=True,inplace=True)
frame.head()

,TICKET_NUMBER,NIGHT_DEPARTURE
0,1.304255e+12,1
1,5.429743e+12,1
2,1.158987e+13,1
3,1.289457e+13,1
4,1.534955e+13,1


Poniżej kod który dodaje do słownika wszystkie odcinki trasy, kluczem jest numer biletu.

In [9]:
x = 0
ticktes_for_frame = []
tickets_list = []
dictionary = []

for number in tickets:
    if number in tickets_list:
        flight1 = "-" + flight_end[x]
        dictionary["AIRPORT_PATH "] += flight1
        ticktes_for_frame += [dictionary]
    if number not in tickets_list:
        flight = flight_start[x] + "-" + flight_end[x]
        dictionary = {"TICKET_NUMBER":number,"AIRPORT_PATH ":flight}
        tickets_list.append(number)
        ticktes_for_frame += [dictionary]
    x += 1

Po przekształceniu słownika na dataframe muszę usunąć duplikaty i zresetować index.

In [10]:
frame2 = pd.json_normalize(ticktes_for_frame)
frame2 = frame2.drop_duplicates(subset="TICKET_NUMBER", keep='first', inplace=False)
frame2.reset_index(drop=True,inplace=True)
frame2.head

<bound method NDFrame.head of           TICKET_NUMBER        AIRPORT_PATH 
0          125352212788  KLI-OUO-TIL-OUO-KLI
1          176790448325          LPF-OUO-LPF
2         1304254658658          ZNT-OUO-ZNT
3         2777327103064          OUO-TWZ-OUO
4         2892534111367          LFF-UIT-TWZ
...                 ...                  ...
19857  9998951172243596          OUO-UGK-OUO
19858  9999144924189102          OUO-ZNT-OUO
19859  9999333249110636          OUO-KDT-OUO
19860  9999544290405665          LFN-OUO-NWZ
19861  9999641990406612              LFF-UGK

[19862 rows x 2 columns]>

In [11]:
frame2.tail()

,TICKET_NUMBER,AIRPORT_PATH
19857,9998951172243596,OUO-UGK-OUO
19858,9999144924189102,OUO-ZNT-OUO
19859,9999333249110636,OUO-KDT-OUO
19860,9999544290405665,LFN-OUO-NWZ
19861,9999641990406612,LFF-UGK


Poniżej kolejny arlgorytm. Tym razem tworze słownik z długością całkowitego lotu dla danego biletu

In [12]:
x = 0
tickets_for_frame = []
tickets_list = []
dictionary = []


for i in tickets:
    number = i
    if number in tickets_list:
        distance = flight_distance[x] 
        dictionary["FLIGHT_DISTANCE_FULL"] += distance
        tickets_for_frame += [dictionary]
    if number not in tickets_list:
        dictionary = {"TICKET_NUMBER":number,"FLIGHT_DISTANCE_FULL":flight_distance[x]}
        tickets_list.append(number)
        tickets_for_frame += [dictionary]
    x += 1

Ponownie konwersja słownika na dataframe i sprawdzenie czy liczba wierszy się zgadza.

In [13]:
frame3 = pd.json_normalize(tickets_for_frame)
frame3 = frame3.drop_duplicates(subset="TICKET_NUMBER", keep='first', inplace=False)
frame3.reset_index(drop=True,inplace=True)
frame3.head

<bound method NDFrame.head of           TICKET_NUMBER  FLIGHT_DISTANCE_FULL
0          125352212788                 19032
1          176790448325                  1046
2         1304254658658                  4474
3         2777327103064                  2774
4         2892534111367                  2272
...                 ...                   ...
19857  9998951172243596                  2204
19858  9999144924189102                  4474
19859  9999333249110636                 18820
19860  9999544290405665                  3488
19861  9999641990406612                  1470

[19862 rows x 2 columns]>

Teraz tworzę dwie listy z kosztów podróży i długości lotów. Dzięki odpowiedniemu sortowaniu, są one dopasowane.

In [14]:
cost_list = []
for i in dataframe_basic.TOTAL_PRICE_PLN:
    cost_list.append(i)

In [15]:
distance_list = []
for i in frame3.FLIGHT_DISTANCE_FULL:
    distance_list.append(i)

Kod dzięki któremu obliczam koszt w PLN za każdy km trasy.

In [16]:
cost_PLN = []
x = 0

for cost in cost_list:
    distance = distance_list[x] 
    cost_now = i
    if cost != 0:
        add = distance/cost
        cost_PLN.append(add)
    if cost == 0:
        cost_PLN.append(0)
    x += 1

Dopasowywuje to do numerów biletu

In [17]:
disctionary = []
x= 0 

for i in frame3.TICKET_NUMBER:
    disctionary.append({"TICKET_NUMBER":i,"PRICE_KM_PLN":cost_PLN[x]})
    x += 1

In [18]:
frame4 = pd.json_normalize(disctionary)
frame4 = frame4.drop_duplicates(subset="TICKET_NUMBER", keep='first', inplace=False)
frame4.reset_index(drop=True,inplace=True)

Nadszedł czas na złączenie wszystkich stworzonych przeze mnie tabel.

In [19]:
final_frames = frame2.merge(frame4, on='TICKET_NUMBER', how="inner").merge(frame, on='TICKET_NUMBER', how="outer")
final_frames = final_frames.drop_duplicates(subset="TICKET_NUMBER", keep='first', inplace=False)
final_frames = final_frames.fillna(0)

In [20]:
final_frames.tail()

,TICKET_NUMBER,AIRPORT_PATH,PRICE_KM_PLN,NIGHT_DEPARTURE
19858,9.999145e+15,OUO-ZNT-OUO,3.257348,1
19859,9.999333e+15,OUO-KDT-OUO,6.878001,1
19860,9.999544e+15,LFN-OUO-NWZ,4.247753,0
19861,9.999642e+15,LFF-UGK,1.163269,0
20094,0.000000e+00,0,0.000000,1


In [21]:
final_frames.drop(20094, axis=0, inplace=True)

In [22]:
dataframe_basic.sort_values(by=["TICKET_NUMBER"],inplace=True)
dataframe_basic.reset_index(drop=True,inplace=True)

In [23]:
final_data_frame = dataframe_basic.merge(final_frames, on='TICKET_NUMBER', how="outer")
final_data_frame = final_data_frame.drop_duplicates(subset="TICKET_NUMBER", keep='first', inplace=False)

Na koniec jeszcze pozbywamy się niepotrzebnych zer, po przecinku.

In [24]:
pd.options.display.float_format = '{:.0f}'.format

In [25]:
final_data_frame.head(10)

,BOOKING_ID,TICKET_NUMBER,ORIGINAL_TICKET_NUMBER,FORM_OF_PAYMENT,CURRENCY,TOTAL_PRICE,TOTAL_PRICE_PLN,PAX_GENDER,PAX_TYPE,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,LOYAL_CUSTOMER_ID,LOYAL_CUSTOMER_DATE_OF_BIRTH,LOYAL_CUSTOMER_REGISTERED_DATE,AIRPORT_PATH,PRICE_KM_PLN,NIGHT_DEPARTURE
0,6693977881414520,125352212788,9379265972706784,CA,EUR,540,2276,M,ADULT,N,N,NaN,NaN,NaN,KLI-OUO-TIL-OUO-KLI,34,0
1,7251381056182327,176790448325,9379265972706784,CCCA5,EUR,188,790,M,ADULT,N,N,NaN,NaN,NaN,LPF-OUO-LPF,2,0
2,9431884934224588,1304254658658,9379265972706784,CCAX3,PLN,2422,2422,F,ADULT,Y,N,NaN,NaN,NaN,ZNT-OUO-ZNT,5,1
3,8901914434731459,2777327103064,9379265972706784,CCCA5,PLN,732,732,F,ADULT,N,N,NaN,NaN,NaN,OUO-TWZ-OUO,6,0
4,9063421527061142,2892534111367,9379265972706784,CCCA5,EUR,132,567,M,ADULT,N,N,NaN,NaN,NaN,LFF-UIT-TWZ,3,0
5,6854442356964813,3536014923107,9379265972706784,CCCA5,SEK,3478,1396,M,ADULT,N,N,NaN,NaN,NaN,UIT-OUO-EIE-OUO-UIT,3,0
6,3215132367206465,3756443774395,9379265972706784,CA,PLN,2308,2308,M,ADULT,N,N,NaN,NaN,NaN,OUO-TIL-OUO,17,0
7,4841315649458329,5054213784705,9379265972706784,CA,EUR,166,709,M,ADULT,N,N,NaN,NaN,NaN,IDP-OUO-IDP,2,0
8,8356981739214475,5429742902077,9379265972706784,CA,EUR,412,1768,F,ADULT,Y,N,NaN,NaN,NaN,KDT-OUO-BUG,12,1
9,9145128903140696,5753901709355,9379265972706784,CCDC3,EUR,156,659,M,ADULT,N,N,NaN,NaN,NaN,VWT-OUO,2,0
